Scenario 2: A cross-functional team with one data scientist working on an ML model
MLflow setup:

tracking server: yes, local server
backend store: sqlite database
artifacts store: local filesystem
The experiments can be explored locally by accessing the local tracking server.

To run this example you need to launch the mlflow server locally by running the following command in your terminal:

mlflow server --backend-store-uri sqlite:///backend.db

In [1]:
import mlflow


mlflow.set_tracking_uri("http://127.0.0.1:5000")

In [7]:
print(f"tracking URI: '{mlflow.get_tracking_uri()}'")

tracking URI: 'http://127.0.0.1:5000'


In [9]:
mlflow.search_experiments()

[<Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716559949916, experiment_id='0', last_update_time=1716559949916, lifecycle_stage='active', name='Default', tags={}>]

In [10]:
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
from sklearn.metrics import accuracy_score

mlflow.set_experiment("my-experiment-1")

with mlflow.start_run():

    X, y = load_iris(return_X_y=True)

    params = {"C": 0.1, "random_state": 42}
    mlflow.log_params(params)

    lr = LogisticRegression(**params).fit(X, y)
    y_pred = lr.predict(X)
    mlflow.log_metric("accuracy", accuracy_score(y, y_pred))

    mlflow.sklearn.log_model(lr, artifact_path="models")
    print(f"default artifacts URI: '{mlflow.get_artifact_uri()}'")

2024/05/24 16:20:57 INFO mlflow.tracking.fluent: Experiment with name 'my-experiment-1' does not exist. Creating a new experiment.


default artifacts URI: '/Users/rajitsanghvi/Library/CloudStorage/OneDrive-OPEDGMBH/General/01_Github/MLOps/02-experiment-tracking/running_mlflow-example/artifacts_local/1/11d9c9f41cc2454a9077070d564d214a/artifacts'


In [11]:
mlflow.search_experiments()

[<Experiment: artifact_location='/Users/rajitsanghvi/Library/CloudStorage/OneDrive-OPEDGMBH/General/01_Github/MLOps/02-experiment-tracking/running_mlflow-example/artifacts_local/1', creation_time=1716560457982, experiment_id='1', last_update_time=1716560457982, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716559949916, experiment_id='0', last_update_time=1716559949916, lifecycle_stage='active', name='Default', tags={}>]

In [14]:
from mlflow.tracking import MlflowClient

client = MlflowClient("http://127.0.0.1:5000")

In [16]:
client.search_registered_models()

[]

In [26]:
client.search_experiments()

[<Experiment: artifact_location='/Users/rajitsanghvi/Library/CloudStorage/OneDrive-OPEDGMBH/General/01_Github/MLOps/02-experiment-tracking/running_mlflow-example/artifacts_local/1', creation_time=1716560457982, experiment_id='1', last_update_time=1716560457982, lifecycle_stage='active', name='my-experiment-1', tags={}>,
 <Experiment: artifact_location='mlflow-artifacts:/0', creation_time=1716559949916, experiment_id='0', last_update_time=1716559949916, lifecycle_stage='active', name='Default', tags={}>]

In [37]:
experiment = client.get_experiment_by_name("my-experiment-1")
runs = client.search_runs(
        experiment_ids=experiment.experiment_id)
runs[0]

<Run: data=<RunData: metrics={'accuracy': 0.96}, params={'C': '0.1', 'random_state': '42'}, tags={'mlflow.log-model.history': '[{"run_id": "11d9c9f41cc2454a9077070d564d214a", '
                             '"artifact_path": "models", "utc_time_created": '
                             '"2024-05-24 14:20:58.518788", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '
                             '"predict_fn": "predict", "loader_module": '
                             '"mlflow.sklearn", "python_version": "3.9.19", '
                             '"env": {"conda": "conda.yaml", "virtualenv": '
                             '"python_env.yaml"}}, "sklearn": '
                             '{"pickled_model": "model.pkl", '
                             '"sklearn_version": "1.4.2", '
                             '"serialization_format": "cloudpickle", "code": '
                             'null}}, "model_uuid": '
                             '"d62e8ec1176

In [38]:
run_id = "11d9c9f41cc2454a9077070d564d214a"
mlflow.register_model(
    model_uri=f"runs:/{run_id}/models",
    name='iris-classifier'
)

Successfully registered model 'iris-classifier'.
2024/05/24 16:34:05 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: iris-classifier, version 1
Created version '1' of model 'iris-classifier'.


<ModelVersion: aliases=[], creation_timestamp=1716561245107, current_stage='None', description='', last_updated_timestamp=1716561245107, name='iris-classifier', run_id='11d9c9f41cc2454a9077070d564d214a', run_link='', source='/Users/rajitsanghvi/Library/CloudStorage/OneDrive-OPEDGMBH/General/01_Github/MLOps/02-experiment-tracking/running_mlflow-example/artifacts_local/1/11d9c9f41cc2454a9077070d564d214a/artifacts/models', status='READY', status_message='', tags={}, user_id='', version='1'>